### Wandb

In [1]:
!pip install --upgrade wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 82.8 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.19.1
    Uninstalling wandb-0.19.1:
      Successfully uninstalled wandb-0.19.1


In [2]:
!pip install python-dotenv

In [3]:
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()

# Lấy key từ biến môi trường
wandb_api_key = os.getenv("WANDB_API_KEY")
print(wandb_api_key[:5])

c8767


In [4]:
import wandb
import os

# Lấy API key từ biến môi trường và đăng nhập
wandb.login(key=wandb_api_key)


wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Inference

Bug:
```bash
model_path = artifact_dir  # Đường dẫn đến mô hình đã tải
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
```

Sửa thành
```bash
# Khởi tạo mô hình trống
from safetensors.torch import load_file
model = BERTIntentClassification(model_name="bert-base-uncased", num_classes=6)
weights_path = os.path.join(artifact_dir, "model.safetensors") # Đường dẫn đến tệp `model.safetensors`
state_dict = load_file(weights_path) # Tải trọng số vào mô hình
model.load_state_dict(state_dict)
model.eval()


# Chuyển mô hình sang chế độ đánh giá
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model loaded and running on device: {device}")
```

### Đánh giá hàng loạt

In [22]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModel
import numpy as np
import torch
import torch.nn as nn
import os
from typing import List
from tqdm import tqdm




In [23]:

class BERTIntentClassification(nn.Module):


    def __init__(self, model_name="bert-base-uncased", num_classes=10, dropout_rate=0.1, cache_dir = "huggingface"):
        super(BERTIntentClassification, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name, cache_dir = cache_dir)
        # Get BERT hidden size
        hidden_size = self.bert.config.hidden_size
        self.ffnn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_classes)
        )


    def freeze_bert(self):
        for param in self.bert.parameters():
            param.requires_grad = False


    def get_pooling(self, hidden_state, attention_mask):
        """
        Get mean pooled representation from BERT hidden states
        Args:
            hidden_state: BERT output containing hidden states
        Returns:
            pooled_output: Mean pooled representation of the sequence
        """
        # Get last hidden state
        last_hidden_state = hidden_state.last_hidden_state  # Shape: [batch_size, seq_len, hidden_size]

        if attention_mask is not None:
            # Expand attention mask to match hidden state dimensions
            attention_mask = attention_mask.unsqueeze(-1)  # [batch_size, seq_len, 1]

            # Mask out padding tokens
            masked_hidden = last_hidden_state * attention_mask

            # Calculate mean (sum / number of actual tokens)
            sum_hidden = torch.sum(masked_hidden, dim=1)  # [batch_size, hidden_size]
            count_tokens = torch.sum(attention_mask, dim=1)  # [batch_size, 1]
            pooled_output = sum_hidden / count_tokens
        else:
            # If no attention mask, simply take mean of all tokens
            pooled_output = torch.mean(last_hidden_state, dim=1)

        return pooled_output


    def forward(self, input_ids, attention_mask, **kwargs):
        """
        Forward pass of the model
        Args:
            input_ids: Input token IDs
            attention_mask: Attention mask for padding
        Returns:
            logits: Raw logits for each class
        """
        # Get BERT hidden states
        hidden_state = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        # Get pooled representation
        hidden_state_pooling = self.get_pooling(hidden_state=hidden_state, attention_mask=attention_mask)

        # Pass through FFNN classifier
        logits = self.ffnn(hidden_state_pooling)

        return logits


In [29]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import wandb
import pandas as pd
import os
import time



# 1. Tải mô hình từ artifact trên WandB
run = wandb.init(project="bert-intent-classification")  # Tên dự án trong WandB
artifact = run.use_artifact('doanngoccuong_nh/bertIntentClassification/best_model_epoch_30:v0', type='model')
artifact_dir = artifact.download()
print("Files in artifact_dir:", os.listdir(artifact_dir))

# Đường dẫn tệp cấu hình
config_path = os.path.join(artifact_dir, "config.json")

# Kiểm tra và cập nhật tệp config.json
config = {
    "model_type": "bert",
    "hidden_size": 768,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "vocab_size": 30522
}
with open(config_path, "w") as f:
    json.dump(config, f, indent=4)
print(f"Config.json updated at {config_path}")


# 2. Create config.json if not available
config = AutoConfig.from_pretrained("bert-base-uncased")

# Tải tokenizer từ mô hình gốc
original_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Lưu các tệp cần thiết vào artifact_dir
original_tokenizer.save_pretrained(artifact_dir)

print(f"Tokenizer files saved to {artifact_dir}")

# 4. Tải mô hình đã lưu và tokenizer
model_path = artifact_dir  # Đường dẫn đến mô hình đã tải
tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForSequenceClassification.from_pretrained(model_path)



# Khởi tạo mô hình trống
from safetensors.torch import load_file
model = BERTIntentClassification(model_name="bert-base-uncased", num_classes=6)
weights_path = os.path.join(artifact_dir, "model.safetensors") # Đường dẫn đến tệp `model.safetensors`
state_dict = load_file(weights_path) # Tải trọng số vào mô hình
model.load_state_dict(state_dict)
model.eval()


# Chuyển mô hình sang chế độ đánh giá
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model loaded and running on device: {device}")




# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
max_seq_length = 512

def preprocess_input(question, answer, tokenizer, max_seq_length):
    """
    Tiền xử lý dữ liệu đầu vào bằng cách ghép nối câu hỏi và câu trả lời với các token đặc biệt.
    """
    input_text = f"[CLS] {question.strip()} [SEP] {answer.strip()} [SEP]"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_seq_length
    )
    return inputs

# 6. Khởi tạo biến lưu kết quả
results = []
correct_predictions = 0

def map_label(pred_class, label_mapping):
    return label_mapping.get(pred_class, f"Unknown (Class ID: {pred_class})")

# Cập nhật label_mapping từ thông tin huấn luyện
label_mapping = {
    0: "intent_fallback",
    1: "intent_learn_more",
    2: "intent_negative",
    3: "intent_neutral",
    4: "intent_positive",
    5: "silence"
}

# 7. Thực hiện inference trên từng dòng dữ liệu
import os
import pandas as pd
import shutil
import torch

def process_and_update_file(input_file, output_file, model, tokenizer, label_mapping, max_seq_length, device, num_rows=None):
    """
    Processes an input Excel file, performs inference, and updates the file with predicted results.

    Args:
        input_file (str): Path to the input Excel file.
        output_file (str): Path to the output Excel file.
        model: The trained model for inference.
        tokenizer: Tokenizer for preprocessing.
        label_mapping (dict): Mapping from class index to label.
        max_seq_length (int): Maximum sequence length for the tokenizer.
        device: PyTorch device (e.g., 'cpu' or 'cuda').
        num_rows (int, optional): Number of rows to process. Default is None (process all rows).
    """
    # Sao chép file gốc nếu file output chưa tồn tại
    if not os.path.exists(output_file):
        shutil.copy(input_file, output_file)
        print(f"File copied from {input_file} to {output_file}")

    # Đọc dữ liệu từ file output
    data = pd.read_excel(output_file)

    # Giới hạn số dòng nếu cần
    if num_rows is not None:
        data = data.head(num_rows)
        print(f"Processing only the first {num_rows} rows.")

    # Xử lý inference và thêm cột mới
    results = []
    correct_predictions = 0



    for idx, row in data.iterrows():
        question = row["robot"]
        answer = row["user_answer"] if not pd.isna(row["user_answer"]) else ""
        true_intent = row["user_intent"]

        # Tiền xử lý đầu vào
        inputs = preprocess_input(question, answer, tokenizer, max_seq_length)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Thực hiện dự đoán
        start_time = time.time()
        with torch.no_grad():
            logits = model(**inputs)  # Custom model directly returns logits
            predicted_class = torch.argmax(logits, dim=1).item()
            predicted_label = map_label(predicted_class, label_mapping)
        end_time = time.time()

        # Calculate response time
        response_time = end_time - start_time

        # Kiểm tra đúng sai
        is_correct = (predicted_label == true_intent)
        if is_correct:
            correct_predictions += 1

        # Lưu kết quả
        results.append({
            "predicted_intent": predicted_label,
            "is_correct": is_correct,
            "model_response_time": response_time
        })

        print(f"Question: {question}")
        print(f"Answer: {answer}")
        print(f"Inputs: {inputs}")
        print(f"Logits: {logits}")
        print(f"Predicted class: {predicted_class}")
        print(f"Predicted label: {predicted_label}")

    # Tạo DataFrame từ kết quả
    results_df = pd.DataFrame(results)

    # Thêm cột vào DataFrame ban đầu
    data["predicted_intent"] = results_df["predicted_intent"]
    data["is_correct"] = results_df["is_correct"]
    data["model_response_time"] = results_df["model_response_time"]

    # Ghi kết quả trở lại file Excel
    with pd.ExcelWriter(output_file, engine="openpyxl", mode="w") as writer:
        data.to_excel(writer, index=False)

    # Tính accuracy
    accuracy = correct_predictions / len(data)
    print(f"Accuracy: {accuracy:.2%}")
    print(f"Evaluation results saved to {output_file}")

# Định nghĩa các tham số cần thiết
input_file = "/content/processed_data_example_v4_15000Data.xlsx"
output_file = "evaluation_results.xlsx"
num_rows = 30  # Số lượng dòng muốn đánh giá
# model = ...  # Model đã huấn luyện
# tokenizer = ...  # Tokenizer tương ứng
# label_mapping = {0: "intent_A", 1: "intent_B", 2: "intent_C"}  # Mapping nhãn
# max_seq_length = 128  # Độ dài tối đa
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Gọi hàm để xử lý và cập nhật file
# Định nghĩa các tham số cần thiết


# Gọi hàm để xử lý và giới hạn số dòng
process_and_update_file(input_file, output_file, model, tokenizer, label_mapping, max_seq_length, device, num_rows=num_rows)




wandb: Downloading large artifact best_model_epoch_30:v0, 419.95MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:1.6


Files in artifact_dir: ['model.safetensors', 'vocab.txt', 'tokenizer_config.json', 'special_tokens_map.json', 'training_args.bin', 'tokenizer.json', 'config.json']
Config.json updated at /content/artifacts/best_model_epoch_30:v0/config.json
Tokenizer files saved to /content/artifacts/best_model_epoch_30:v0
Model loaded and running on device: cuda
Processing only the first 30 rows.
Question: Được rồi, bây giờ chúng ta sẽ chơi một trò chơi! Hãy kể tên nhiều từ thuộc cùng 1 chủ đề nhé. Chủ đề lần này là hành động bắt đầu bằng từ "eat food". Tớ ví dụ nhé, "eat pizza", đến lượt cậu nhé
Answer: Tớ ăn cơm.
Inputs: {'input_ids': tensor([[  101,   101,  1102, 19098,  2278, 25223,  1010,  3016, 21025,  2080,
         15972, 11937,  7367, 18151,  9587,  2102, 19817,  2080, 18151,   999,
         10974, 17710,  2702, 18699, 17301, 10722, 16215, 19098,  2278, 12731,
          3070,  1015, 14684,  1102,  2063, 18699,  2063,  1012, 14684,  1102,
          2063, 17595, 29349,  2474,  7658,  2232,  110